In [1]:
# нагрузка по работе не позволяет выполнить до конца работу, однако скоро решение будет закомиченно.  
# 

In [ ]:
#Домашнее задание по теме «Рекомендации на основе содержания»
#Преподаватель: Александр Кондрашкин
#
#    1. Использовать dataset MovieLens
#    2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:
#
#    3. TF-IDF на тегах и жанрах
#    4. Средние оценки (+ median, variance, etc.) пользователя и фильма
#
#    5. Оценить RMSE на тестовой выборке



In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Фичи будут складываться из tf-idf на жанрах и тегах,  

In [3]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)